# Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы)

Получившийся список должен содержать в себе минимум:

* Наименование вакансии.
* Предлагаемую зарплату (отдельно минимальную и максимальную).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). 
Структура должна быть одинаковая для вакансий с обоих сайтов. 

Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

### Import libraries

In [ ]:
import requests
import re
import pandas as pd
import json

from bs4 import BeautifulSoup as bs

### Define helper functions

In [ ]:
def getCompensationCurrency(str):
  return str.split()[-1]

def getMinCompensation(str):
  numbers = re.findall(r'\d+', str.replace(' ', ''))
  if 'до' in str:
    return None
  else:
    return int(numbers[-1]) if 'от' in str else int(numbers[0])

def getMaxCompensation(str):
  numbers = re.findall(r'\d+', str.replace(' ', ''))
  if 'от' in str:
    return None
  else:
    return int(numbers[0]) if 'до' in str else int(numbers[-1])

In [ ]:
def parseJob(job):
  title = job.find('a', {'class': 'bloko-link'}).getText()
  link = job.find('a', {'class': 'bloko-link'}).get('href')

  compensation = job.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
  compensationText = None if compensation is None else compensation.getText().replace(u'\u202f', ' ')
  
  minCompensation = None if compensationText is None else getMinCompensation(compensationText)
  maxCompensation = None if compensationText is None else getMaxCompensation(compensationText)
  compensationCurrency = None if compensationText is None else getCompensationCurrency(compensationText)

  return {
      'title': title,
      'link': link,
      'minCompensation': minCompensation,
      'maxCompensation': maxCompensation,
      'compensationCurrency': compensationCurrency
  }

### Get the search request

In [ ]:
searchRequest = input('Enter a search request: ')

Enter a search request: skala js


### Make a request

In [ ]:
fullJobsList = []

In [ ]:
baseUrl = 'https://hh.ru'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0'
}

params = {
    'area': '1',
    'fromSearchLine': 'true',
    'st': 'searchVacancy',
    'text': searchRequest
}

url = baseUrl + '/search/vacancy'

while True:
  response = requests.get(url, params=params, headers=headers)
  soup = bs(response.text, 'html.parser')

  jobs = soup.find_all('div', {'class': 'vacancy-serp-item'})

  parsedJobs = list(map(lambda x: parseJob(x), jobs))
  fullJobsList.extend(parsedJobs)

  nextButton = soup.find('a', {'data-qa': 'pager-next'})
  url = None if nextButton is None else baseUrl + nextButton.get('href')

  params = None
  
  if url is None:
    break

### Convert results to Pandas DataFrame

In [ ]:
df = pd.DataFrame(fullJobsList)
df

,title,link,minCompensation,maxCompensation,compensationCurrency
0,Java-программист,https://hh.ru/vacancy/46158099?from=vacancy_se...,200000,300000,руб.


### Save results as JSON

In [ ]:
with open('hh.json', 'w') as fout:
    json.dump(fullJobsList , fout)